In [7]:
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import pandas as pd


# B-S portfolio_pricer

In [37]:
def black_scholes_option_pricer(S, K, T, r, q, sigma, option_type='call'):
   
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    else:
        raise ValueError("Type d'option non valide. Utilisez 'call' ou 'put'.")

    return option_price

def option_greeks(S, K, T, r, q, sigma, option_type):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    N_prime_d1 = norm.pdf(d1)

    delta_call = 0
    delta_put = 0
    gamma_call = 0
    gamma_put = 0
    vega_call = 0
    vega_put = 0
    theta_call = 0
    theta_put = 0

    if option_type == 'call':
        delta_call = norm.cdf(d1)
        gamma_call = N_prime_d1 / (S * sigma * np.sqrt(T))
        vega_call = S * np.sqrt(T) * N_prime_d1
        theta_call = -(S * sigma * N_prime_d1) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        delta_put = norm.cdf(d1) - 1
        gamma_put = N_prime_d1 / (S * sigma * np.sqrt(T))
        vega_put = S * np.sqrt(T) * N_prime_d1
        theta_put = -(S * sigma * N_prime_d1) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)

    return delta_call, delta_put, gamma_call, gamma_put, vega_call, vega_put, theta_call, theta_put

def portfolio_option_pricer(options):
    total_price = 0
    total_delta_call = 0
    total_delta_put = 0
    total_gamma_call = 0
    total_gamma_put = 0
    total_vega_call = 0
    total_vega_put = 0
    total_theta_call = 0
    total_theta_put = 0

    for option in options:
        option_price = black_scholes_option_pricer(option['S'], option['K'], option['T'],
                                                   option['r'], option['q'], option['sigma'], option['type'])
        total_price += option_price * option['quantity']

        delta_call, delta_put, gamma_call, gamma_put, vega_call, vega_put, theta_call, theta_put = option_greeks(option['S'], option['K'], option['T'],
                                                  option['r'], option['q'], option['sigma'], option['type'])

        total_delta_call += delta_call * option['quantity']
        total_delta_put += delta_put * option['quantity']
        total_gamma_call += gamma_call * option['quantity']
        total_gamma_put += gamma_put * option['quantity']
        total_vega_call += vega_call * option['quantity']
        total_vega_put += vega_put * option['quantity']
        total_theta_call += theta_call * option['quantity']
        total_theta_put += theta_put * option['quantity']

    return total_price, total_delta_call, total_delta_put, total_gamma_call, total_gamma_put, total_vega_call, total_vega_put, total_theta_call, total_theta_put

# Exemple d'utilisation
options_portfolio = [
    {'S': 100, 'K': 110, 'T': 1, 'r': 0.05, 'q' : 0.07, 'sigma': 0.2, 'type': 'call', 'quantity': 2},
    {'S': 100, 'K': 90, 'T': 1, 'r': 0.05, 'q' : 0.07, 'sigma': 0.2, 'type': 'put', 'quantity': 1},
    # Ajoutez d'autres options au besoin
]

total_portfolio_price, total_portfolio_delta_call, total_portfolio_delta_put, total_portfolio_gamma_call, total_portfolio_gamma_put, total_portfolio_vega_call, total_portfolio_vega_put, total_portfolio_theta_call, total_portfolio_theta_put = portfolio_option_pricer(options_portfolio)


print(f"Prix total du portefeuille d'options : {total_portfolio_price:.2f}")
print(f"Delta_call total du portefeuille : {total_portfolio_delta_call:.4f}")
print(f"Gamma_call total du portefeuille : {total_portfolio_gamma_call:.4f}")
print(f"Vega_call total du portefeuille : {total_portfolio_vega_call:.4f}")
print(f"Theta_call total du portefeuille : {total_portfolio_theta_call:.4f}")
print(f"Delta_put total du portefeuille : {total_portfolio_delta_put:.4f}")
print(f"Gamma_put total du portefeuille : {total_portfolio_gamma_put:.4f}")
print(f"Vega_put total du portefeuille : {total_portfolio_vega_put:.4f}")
print(f"Theta_put total du portefeuille : {total_portfolio_theta_put:.4f}")

Prix total du portefeuille d'options : 10.85
Delta_call total du portefeuille : 0.6337
Gamma_call total du portefeuille : 0.0356
Vega_call total du portefeuille : 71.2239
Theta_call total du portefeuille : -9.7314
Delta_put total du portefeuille : -0.2992
Gamma_put total du portefeuille : 0.0174
Vega_put total du portefeuille : 34.7254
Theta_put total du portefeuille : -1.8806
